In [1]:
import pandas as pd

In [24]:
rawdata = pd.read_csv("purchase_data.csv")
rawdata.head()
rawdata = rawdata.rename(columns={'Purchase ID':'purchase_id',
                                    'SN':'screenname',
                                    'Age':'age',
                                    'Gender':'gender',
                                    'Item ID':'item_id',
                                    'Item Name':'item_name',
                                    'Price':'price'})
rawdata.head()

,purchase_id,screenname,age,gender,item_id,item_name,price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


# Player Count

In [3]:
player_count_df = pd.DataFrame({
    "Total Players" : rawdata.screenname.nunique()
}, index=[0])

player_count_df

,Total Players
0,576


# Purchasing Analysis (Total)

In [21]:

purchasing_analysis_df = pd.DataFrame({
    "Number of Unique Items" : rawdata.item_id.nunique(), 
    "Average Price" :rawdata.price.mean(),
    "Number of Purchases" : rawdata.purchase_id.nunique(),
    "Total Revenue" : rawdata.price.sum()
}, index = [""])

purchasing_analysis_df["Average Price"] = purchasing_analysis_df["Average Price"].astype(float).map("${:,.2f}".format)
purchasing_analysis_df["Total Revenue"] = purchasing_analysis_df["Total Revenue"].astype(float).map("${:,.2f}".format)

purchasing_analysis_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
,179,$3.05,780,"$2,379.77"


# Gender Demographics

In [5]:
gen_dem_df = pd.DataFrame({
    "Total Count" : rawdata[["screenname", "gender"]].drop_duplicates().gender.value_counts(), 
    "Percentage of Players" : rawdata[["screenname", "gender"]].drop_duplicates().gender.value_counts(normalize = True)
    .astype(float).map("{:.2%}".format)
})

gen_dem_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


# Purchasing Analysis (Gender)

In [6]:
gen_purch_df = pd.DataFrame({
    "Purchase Count" : rawdata.groupby("gender")["purchase_id"].count(),
    "Average Purchase Price" : rawdata.groupby("gender")["price"].mean().astype(float).map("${:,.2f}".format), 
    "Total Purchase Value" : rawdata.groupby("gender")["price"].sum().astype(float).map("${:,.2f}".format), 
    "Average Purchase Total per Person" : (rawdata.groupby("gender")["price"].sum() / 
    rawdata.groupby("gender")["screenname"].nunique()).astype(float).map("${:,.2f}".format)
})

gen_purch_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


# Age Demografics

The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

Purchase Count (done)
Average Purchase Price (done)
Total Purchase Value (done)
Average Purchase Total per Person by Age Group

In [7]:
#I know the max is 45, but this way the program will work even if older people buy things
age_bins = [0, 9.9, 14.9, 19.9, 24.9, 29.9, 34.9, 39.9, 44.9, 49.9, 54.9, 59.9,\
            64.9, 69.9, 74.9, 79.9, 84.9, 89.9, 94.9, 99.9, 99999]
age_groups = ["Less than 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49", \
              "50-54", "55-59", "60-64", "65-69", "70-74", "75-79", "80-84", "85-89", "90-94", "94-99", "Over 100"]

In [8]:
rawdata["age_range"] = pd.cut(rawdata["age"], age_bins, labels = age_groups, include_lowest = True)
rawdata.head()

,purchase_id,screenname,age,gender,item_id,item_name,price,age_range
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53,20-24
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56,40-44
2,2,Ithergue48,24,Male,92,Final Critic,4.88,20-24
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27,20-24
4,4,Iskosia90,23,Male,131,Fury,1.44,20-24


In [19]:
age_dem_df = pd.DataFrame({
    "Purchase Count" : rawdata.groupby("age_range")["purchase_id"].count(),
    "Average Purchase Price" : rawdata.groupby("age_range")["price"].mean().astype(float).map("${:,.2f}".format), 
    "Total Purchase Value" : rawdata.groupby("age_range")["price"].sum().astype(float).map("${:,.2f}".format), 
    "Average Purchase Total per Person" : (rawdata.groupby("age_range")["price"].sum() / 
    rawdata.groupby("age_range")["screenname"].nunique()).astype(float).map("${:,.2f}".format)
})

age_dem_df = age_dem_df.dropna(axis = 0, how = "all", )
age_dem_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
age_range,,,,
Less than 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40-44,12,$3.04,$36.54,$3.32
45-49,1,$1.70,$1.70,$1.70


In [10]:
# totalbyage_df = rawdata.groupby(['Age Range']).sum()
# totalbyage_df = totalbyage_df.rename(columns = {"Price" : "Total Price By Age"})
# averagebyage_df = rawdata.groupby(['Age Range']).mean()
# averagebyage_df = averagebyage_df.rename(columns = {"Price" : "Average Price By Age"})
# countbyage_df = rawdata.groupby(['Age Range']).count()
# countbyage_df = countbyage_df.rename(columns = {"Price" : "Purchase Count By Age"})
# snbyage_df = rawdata.groupby(['Age Range'])['SN'].nunique()
# snbyage_df

In [11]:
# totalbyage_df = totalbyage_df[['Total Price By Age']]
# averagebyage_df = averagebyage_df[['Average Price By Age']]
# countbyage_df = countbyage_df[['Purchase Count By Age']]
# countbyage_df

In [12]:
# rawdata = rawdata.set_index("Age Range")
# rawdata.head()

In [13]:
# lessrawdata = pd.merge(rawdata, totalbyage_df, on = "Age Range", how = "outer")
# lessrawdata = pd.merge(lessrawdata, averagebyage_df, on = "Age Range", how = "outer")
# lessrawdata = pd.merge(lessrawdata, countbyage_df, on = "Age Range", how = "outer")
# lessrawdata.head()

In [14]:
# lessrawdata = lessrawdata.dropna(how = "any")
# lessrawdata

# Top Spenders



Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

SN
Purchase Count
Average Purchase Price
Total Purchase Value

In [23]:
findingtopspenders = rawdata.groupby(['sn'], as_index = False).sum()
findingtopspenders.head()

KeyError: 'sn'

In [16]:
pricetopspenders = findingtopspenders.sort_values("Price", ascending = False)
pricetopspenders["Price"] = pricetopspenders["Price"].map("${:.2f}".format)
pricetopspenders = pricetopspenders.reset_index()
pricetopspenders.head()

In [17]:
spenderone = pricetopspenders.iloc([0, 0])
spenderone

In [18]:
findmoretopspenders = rawdata.groupby(['SN'], as_index = False).count()
findmoretopspenders.head()